In [35]:
import seaborn as sns
import csv
import optuna
import optuna.integration.lightgbm as lgb
import itertools, logging, joblib
import numerox as nx
import numerapi
sys.path.append("../../"); import auth_info
NOTE_NAME = "lgbm2"

In [2]:
ROUND = "237"
TOURNAMENT_NAME = "kazutsugi"
TARGET_NAME = f"target_{TOURNAMENT_NAME}"
PREDICTION_NAME = f"prediction_{TOURNAMENT_NAME}"
N_FEATURE = 310

In [3]:
# Load Dataset
def read_csv(file_path):
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
    dtypes = {x: np.float16 for x in column_names if x.startswith(('feature', 'target'))}
    df = pd.read_csv(file_path, dtype=dtypes, index_col=0)
    return df

train = read_csv("../../dataset/numerai_dataset_237/numerai_training_data.csv")
tournament = read_csv("../../dataset/numerai_dataset_237/numerai_tournament_data.csv")

# reform
train.sort_index(axis=1, inplace=True)
tournament.sort_index(axis=1, inplace=True)
cols_feature = train.columns[train.columns.str.contains("feature_")]
cols_target  = train.columns[train.columns.str.contains("target_")]

In [5]:
# load models
base_models = [joblib.load(f"{NOTE_NAME}/best_model_{i}.pkl") for i in range(0,6)]
top_model = joblib.load(f"{NOTE_NAME}/ensemble_lgb.pkl")

In [11]:
# generate signals
signals = pd.concat(
    [pd.Series(m.predict(train.loc[:, m.feature_name()]), index=train.index, name=f"model_{i}") 
     for i,m in enumerate(base_models)], axis=1)
signals["pred"] = top_model.predict(signals)

val_signals = pd.concat(
    [pd.Series(m.predict(tournament.loc[:, m.feature_name()]), index=tournament.index, name=f"model_{i}") 
     for i,m in enumerate(base_models)], axis=1)
val_signals["pred"] = top_model.predict(val_signals)

In [12]:
val_signals.head(1)

,model_0,model_1,model_2,model_3,model_4,model_5,pred
id,,,,,,,
n0003aa52cab36c2,0.491723,0.472704,0.493346,0.493126,0.495358,0.495909,0.483723


In [28]:
tournament[["era", "data_type"]].join(val_signals.pred.rename(PREDICTION_NAME))

,era,data_type,prediction_kazutsugi
id,,,
n0003aa52cab36c2,era121,validation,0.483723
n000920ed083903f,era121,validation,0.487245
n0038e640522c4a6,era121,validation,0.535500
n004ac94a87dc54b,era121,validation,0.504724
n0052fe97ea0c05f,era121,validation,0.495365
...,...,...,...
nffd239464395ebf,eraX,live,0.496788
nffd89078dbed0c8,eraX,live,0.518779
nffe7b410a2e9866,eraX,live,0.496468


In [32]:
# generate prediction csv
val_signals.pred.rename("prediction").to_frame().to_csv(f"{NOTE_NAME}/preds.csv")

In [ ]:
nx.upload(f'{NOTE_NAME}/preds.csv', TOURNAMENT_NAME, auth_info.PUBLIC_KEY, auth_info.PRIVATE_KEY, 
          model_id=auth_info.MODEL_ID[NOTE_NAME])

2020-11-19 10:50:56,544 INFO numerapi.base_api: uploading predictions...
2020-11-19 10:51:09,271 ERROR numerapi.base_api: invalid submission ids. ids must match current tournament data exactly, including ordering. make sure you are using the latest tournament data


Upload exception - invalid submission ids. ids must match current tournament data exactly, including ordering. make sure you are using the latest tournament data
